### Importazione librerie

In [1]:
import os
import requests
import zipfile
import pandas as pd
from io import BytesIO
import ccxt
import time

### Download dati order book da Binance

In [ ]:
def download_and_extract_zip(url, extract_to='.'):
    response = requests.get(url)
    if response.status_code == 200:
        with zipfile.ZipFile(BytesIO(response.content)) as thezip:
            thezip.extractall(extract_to)
    else:
        print(f"Errore nello scaricare {url}: Codice {response.status_code}")

def process_order_book_data(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print(f"Errore nel processare {file_path}: {e}")
        return None

In [3]:
# Parametri
year = 2024
months = range(1, 13)
base_url = 'https://data.binance.vision/data/futures/um/daily/bookDepth/BTCUSDT/'
output_csv = 'BTCUSDT_order_book_2024.csv'

# Lista per contenere tutti i DataFrame
all_data = []

# Scarica, estrai e processa i dati per ogni giorno del 2024
for month in months:
    for day in range(1, 32):
        date_str = f'{year}-{month:02d}-{day:02d}'
        zip_filename = f'BTCUSDT-bookDepth-{date_str}.zip'
        url = f'{base_url}{zip_filename}'
        extract_path = f'./data/{date_str}/'
        os.makedirs(extract_path, exist_ok=True)
        
        try:
            download_and_extract_zip(url, extract_to=extract_path)
            csv_filename = zip_filename.replace('.zip', '.csv')
            file_path = os.path.join(extract_path, csv_filename)
            
            if os.path.exists(file_path):
                df = process_order_book_data(file_path)
                if df is not None:
                    all_data.append(df)
            else:
                print(f"File non trovato: {file_path}")
        except Exception as e:
            print(f"Errore nel processare {date_str}: {e}")

# Concatenare tutti i DataFrame in uno solo
if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    combined_df.to_csv(output_csv, index=False)
    print(f"Dati concatenati salvati in {output_csv}")
else:
    print("Nessun dato da concatenare.")

Errore nel scaricare https://data.binance.vision/data/futures/um/daily/bookDepth/BTCUSDT/BTCUSDT-bookDepth-2024-02-30.zip: Codice 404
File non trovato: ./data/2024-02-30/BTCUSDT-bookDepth-2024-02-30.csv
Errore nel scaricare https://data.binance.vision/data/futures/um/daily/bookDepth/BTCUSDT/BTCUSDT-bookDepth-2024-02-31.zip: Codice 404
File non trovato: ./data/2024-02-31/BTCUSDT-bookDepth-2024-02-31.csv
Errore nel scaricare https://data.binance.vision/data/futures/um/daily/bookDepth/BTCUSDT/BTCUSDT-bookDepth-2024-04-18.zip: Codice 404
File non trovato: ./data/2024-04-18/BTCUSDT-bookDepth-2024-04-18.csv
Errore nel scaricare https://data.binance.vision/data/futures/um/daily/bookDepth/BTCUSDT/BTCUSDT-bookDepth-2024-04-31.zip: Codice 404
File non trovato: ./data/2024-04-31/BTCUSDT-bookDepth-2024-04-31.csv
Errore nel scaricare https://data.binance.vision/data/futures/um/daily/bookDepth/BTCUSDT/BTCUSDT-bookDepth-2024-06-31.zip: Codice 404
File non trovato: ./data/2024-06-31/BTCUSDT-bookDepth-

### Preprocessing order_book_df

#### Lettura order_book_df

In [2]:
# Leggiamo il CSV e convertiamo la colonna "timestamp" in datetime
order_book_df = pd.read_csv('BTCUSDT_order_book_2024.csv', parse_dates=['timestamp'])
# Poiché lo snapshot ha precisione al secondo, prendiamo i timestamp unici
unique_timestamps = sorted(order_book_df['timestamp'].unique())
print(f"Timestamp unici trovati: {len(unique_timestamps)}")
order_book_df

Timestamp unici trovati: 1047474


,timestamp,percentage,depth,notional
0,2024-01-01 00:00:10,-5,11063.461,4.573793e+08
1,2024-01-01 00:00:10,-4,9238.593,3.835965e+08
2,2024-01-01 00:00:10,-3,7446.324,3.103917e+08
3,2024-01-01 00:00:10,-2,5563.144,2.326627e+08
4,2024-01-01 00:00:10,-1,2115.404,8.904219e+07
...,...,...,...,...
10474735,2024-12-31 23:59:31,1,982.935,9.233265e+07
10474736,2024-12-31 23:59:31,2,1719.843,1.623170e+08
10474737,2024-12-31 23:59:31,3,2564.155,2.432439e+08
10474738,2024-12-31 23:59:31,4,3128.343,2.978457e+08


#### Preprocessing order_book_df

In [4]:
# Creiamo il pivot per 'depth'
# pivot_depth = order_book_df.pivot(index='timestamp', columns='percentage', values='depth')
# pivot_depth.columns.name = None  # Rimuove il nome 'percentage' dall'indice delle colonne
# # Rinominiamo le colonne: ad esempio, il livello -5 diventerà 'depth_-5'
# pivot_depth = pivot_depth.rename(columns=lambda x: f'depth_{x}')

# Creiamo il pivot per 'notional'
pivot_notional = order_book_df.pivot(index='timestamp', columns='percentage', values='notional')
pivot_notional.columns.name = None  # Rimuove il nome 'percentage'
pivot_notional = pivot_notional.rename(columns=lambda x: f'notional_{x}')

# Concatenare i due pivot table lungo le colonne
# order_book_agg = pd.concat([pivot_depth, pivot_notional], axis=1).reset_index()
order_book_agg = pd.concat([pivot_notional], axis=1).reset_index()

# Visualizziamo le prime righe per controllo
order_book_agg

,timestamp,notional_-5,notional_-4,notional_-3,notional_-2,notional_-1,notional_1,notional_2,notional_3,notional_4,notional_5
0,2024-01-01 00:00:10,4.573793e+08,3.835965e+08,3.103917e+08,2.326627e+08,8.904219e+07,6.138488e+07,1.655435e+08,2.243606e+08,3.094913e+08,3.338707e+08
1,2024-01-01 00:00:39,4.698861e+08,3.965018e+08,3.234177e+08,2.466879e+08,1.037588e+08,7.441209e+07,1.769790e+08,2.354876e+08,3.209383e+08,3.454925e+08
2,2024-01-01 00:01:02,4.672116e+08,3.951836e+08,3.222620e+08,2.454034e+08,9.290719e+07,7.773906e+07,1.954975e+08,2.438128e+08,3.240160e+08,3.507592e+08
3,2024-01-01 00:01:30,4.652296e+08,3.921109e+08,3.198639e+08,2.443515e+08,9.257850e+07,7.688565e+07,1.955076e+08,2.435765e+08,3.230847e+08,3.591870e+08
4,2024-01-01 00:02:02,4.618363e+08,3.890863e+08,3.170328e+08,2.353286e+08,9.216747e+07,7.944649e+07,1.977384e+08,2.455497e+08,3.251416e+08,3.612228e+08
...,...,...,...,...,...,...,...,...,...,...,...
1047469,2024-12-31 23:57:31,8.617161e+08,7.436043e+08,4.669481e+08,3.129128e+08,1.656252e+08,9.856389e+07,1.738287e+08,2.545384e+08,3.091885e+08,3.754364e+08
1047470,2024-12-31 23:58:01,8.624637e+08,7.425072e+08,4.755094e+08,3.124196e+08,1.654890e+08,9.721174e+07,1.730429e+08,2.523438e+08,3.069253e+08,3.734684e+08
1047471,2024-12-31 23:58:30,8.648124e+08,7.454401e+08,4.778244e+08,3.149477e+08,1.676078e+08,9.718822e+07,1.711980e+08,2.523917e+08,3.050670e+08,3.734145e+08
1047472,2024-12-31 23:59:02,8.540163e+08,7.339014e+08,4.695423e+08,3.076486e+08,1.622543e+08,9.501509e+07,1.647492e+08,2.450605e+08,2.997077e+08,3.664387e+08


### Integrazione dei dati classici sulle candele

In [5]:
candles_df = pd.read_csv('Bitcoin (USDT) 2024 timeframe 1m.csv')
candles_df.set_index('timestamp', inplace=True)
candles_df.index = pd.to_datetime(candles_df.index)
candles_df

,open,high,low,close,volume
timestamp,,,,,
2024-01-01 00:00:00,42284.00,42300.00,42260.01,42300.00,15.059749
2024-01-01 00:01:00,42300.00,42321.99,42299.99,42320.28,4.240722
2024-01-01 00:02:00,42320.28,42335.79,42317.30,42322.57,5.671511
2024-01-01 00:03:00,42322.57,42370.00,42322.57,42366.50,6.801620
2024-01-01 00:04:00,42366.50,42399.68,42366.50,42399.68,6.264730
...,...,...,...,...,...
2024-12-30 23:56:00,92816.59,92878.51,92789.85,92870.74,6.900125
2024-12-30 23:57:00,92870.74,92895.11,92838.44,92844.67,6.369311
2024-12-30 23:58:00,92844.67,92845.68,92797.10,92831.16,6.396956


In [ ]:
# Funzione per scaricare OHLCV per timestamp specifici
def download_ohlcv_for_timestamps(cryptos, market, timeframe_download, timestamps):
    """
    Scarica le candele OHLCV per ogni timestamp presente nella lista 'timestamps',
    utilizzando l'API di ccxt per il mercato specificato.
    """
    exchange_class = getattr(ccxt, market)
    # Abilitiamo il rate limit per evitare errori
    exchange = exchange_class({'enableRateLimit': True})
    exchange.load_markets()
    
    cryptos_pairs = [f"{crypto}/USDT" for crypto in cryptos]
    data_frames = {}
    
    for pair in cryptos_pairs:
        if pair not in exchange.markets:
            raise ValueError(f"Il mercato {pair} non è disponibile su {market}.")
        print(f"Scarico i dati OHLCV per {pair} per {len(timestamps)} timestamp...")
        ohlcv = []
        # Per ogni timestamp, convertiamo in ms ed eseguiamo una chiamata con limit=1
        for ts in timestamps:
            ts_ms = int(pd.Timestamp(ts).timestamp() * 1000)
            try:
                # La chiamata fetch_ohlcv con limit=1 dovrebbe restituire la candela per quel secondo
                batch = exchange.fetch_ohlcv(pair, timeframe_download, ts_ms, limit=1)
            except Exception as e:
                print(f"Errore per timestamp {ts}: {e}")
                continue
            if batch:
                candle = batch[0]
                # Convertiamo il timestamp della candela per verificarne la corrispondenza
                candle_ts = pd.to_datetime(candle[0], unit='ms')
                # In alcuni casi la candela restituita potrebbe non corrispondere esattamente,
                # ma qui l'assumiamo corretta se coincide con il timestamp richiesto.
                if candle_ts == pd.Timestamp(ts):
                    ohlcv.append(candle)
                else:
                    # Se non combaciano, lo segnaliamo e comunque aggiungiamo la candela
                    print(f"Attenzione: candela per {ts} restituita con timestamp {candle_ts}")
                    ohlcv.append(candle)
            else:
                print(f"Nessun dato per il timestamp {ts}")
            # Rispettiamo il rate limit
            # time.sleep(exchange.rateLimit / 1000)
        # Creiamo un DataFrame dai dati raccolti
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        data_frames[pair] = df
    # Ritorna un dizionario con i dati per ogni crypto, eliminando '/USDT' dal nome
    return {k.replace('/USDT', ''): v for k, v in data_frames.items()}

In [ ]:
# Scarica i dati OHLCV per i timestamp dell'order book
cryptos = ['BTC']
market = 'binance'
timeframe_download = '1s'  # Precisione a 1 secondo
plot = False  # Non usato in questo esempio

print("Inizio download OHLCV per timestamp specifici...")
data = download_ohlcv_for_timestamps(cryptos, market, timeframe_download, unique_timestamps)
btc_df = data['BTC']

# Visualizza
btc_df

Inizio download OHLCV per timestamp specifici...
Scarico i dati OHLCV per BTC/USDT per 1047474 timestamp...


KeyboardInterrupt: 